In [6]:
# @title 🛠️ Fast Environment Setup
# @markdown This cell uses pre-compiled binaries to save time. (~45 seconds)

# 1. Fix the 'jedi' dependency error and install server tools
!pip install jedi huggingface_hub pyngrok nest_asyncio fastapi uvicorn -q

# 2. Install pre-compiled llama-cpp-python with CUDA support
# This skips the 5-minute "building wheel" process
!pip install llama-cpp-python[server] --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122 -q

import nest_asyncio
from huggingface_hub import hf_hub_download
from pyngrok import ngrok
import threading
import subprocess

nest_asyncio.apply()

In [7]:
REPO_ID = "tspec2/redstring-8gb"
FILENAME = "meta-llama-3.1-8b.Q8_0.gguf" 

print(f"Fetching {FILENAME} from {REPO_ID}...")

model_path = hf_hub_download(
    repo_id=REPO_ID,
    filename=FILENAME
)

print(f"Model cached at: {model_path}")

Fetching meta-llama-3.1-8b.Q8_0.gguf from tspec2/redstring-8gb...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


meta-llama-3.1-8b.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Model cached at: /root/.cache/huggingface/hub/models--tspec2--redstring-8gb/snapshots/15408353b64155f2a0b79ca513812aa9b38eb7e0/meta-llama-3.1-8b.Q8_0.gguf


In [8]:
# @title Start the API Server
NGROK_TOKEN = "39pFdrEYO2brshKa669babIg7Nr_5pY9xXUPgS7rumDHdeu5Y"

!ngrok config add-authtoken {NGROK_TOKEN}
ngrok.kill()
public_url = ngrok.connect(8000).public_url

def start_api():
    cmd = f"python3 -m llama_cpp.server --model {model_path} --n_gpu_layers -1 --n_ctx 8192 --host 0.0.0.0 --port 8000"
    subprocess.run(cmd, shell=True)

threading.Thread(target=start_api, daemon=True).start()
print(f"\n✨ API IS LIVE! ✨")
print(f"Copy this into your local app: {public_url}/v1")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                

✨ API IS LIVE! ✨
Copy this into your local app: https://inerasably-unpromotive-branda.ngrok-free.dev/v1
